sounddevice, soundfile python libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import librosa
import IPython.display as ipd
from scipy.fft import fft, ifft, fftfreq
import scipy.signal as sig

In [ ]:
filename = librosa.example('nutcracker')
y1, sr1 = librosa.load(filename)
y2, sr2 = librosa.load("bzz.mp3")
y3, sr3 = librosa.load("drone_stock.wav")
y4, sr4 = librosa.load("drone_felix.mp3")
y5, sr5 = librosa.load("movement2_Scarlett 18i20 3rd Gen Multichannel_capture_AUX0.wav",sr=48000)
y6, sr6 = librosa.load("movement2_Scarlett 18i20 3rd Gen Multichannel_capture_AUX1.wav",sr=48000)
y7, sr7 = librosa.load("movement2_Scarlett 18i20 3rd Gen Multichannel_capture_AUX2.wav",sr=48000)
y8, sr8 = librosa.load("movement2_Scarlett 18i20 3rd Gen Multichannel_capture_AUX3.wav",sr=48000)

In [ ]:
file = "static5"

y0, sr = librosa.load(file+"_Scarlett 18i20 3rd Gen Multichannel_capture_AUX0.wav",sr=48000)
y1, sr = librosa.load(file+"_Scarlett 18i20 3rd Gen Multichannel_capture_AUX1.wav",sr=48000)
y2, sr = librosa.load(file+"_Scarlett 18i20 3rd Gen Multichannel_capture_AUX2.wav",sr=48000)
y3, sr = librosa.load(file+"_Scarlett 18i20 3rd Gen Multichannel_capture_AUX3.wav",sr=48000)

In [ ]:
sr4

In [ ]:
sr6

In [ ]:
ipd.Audio(y4,rate=2*sr4)

In [ ]:
N = 50000
t = np.arange(0,N,1)
T = 1000
A = 0
signal0 = np.zeros(N)
signal1 = A*(np.sin(t/T))
signal2 = A*(1/np.sqrt(5)*np.sin(t/T)+2/np.sqrt(5)*np.cos(3*t/T))
signal3 = A*(np.sin(np.sqrt(t)/T))
signal4 = A*(np.sin(t**2/T))
signal5 = y1
signal6 = y2
signal7 = y3
signal8 = y4
noise = np.random.normal(0,1,N)
noise1 = A*np.random.normal(0,1,N)
noise2 = A*np.random.normal(0,1,N)
noise_cauchy = np.random.standard_cauchy(N)


source = noise

time_start = 0*int(sr4*5)
time_duration = int(sr4*0.1)
time_diff = sr4//100
received_1 = source[time_start:time_start+time_duration]+noise1[time_start:time_start+time_duration]
received_2 = source[time_start+time_diff:time_start+time_duration+time_diff]+noise2[time_start+time_diff:time_start+time_duration+time_diff]
received_0 = y0[time_start:time_start+time_duration]
#received_1 = y1[time_start:time_start+time_duration]
#received_2 = y2[time_start:time_start+time_duration]
received_3 = y3[time_start:time_start+time_duration]

lo = 1000
hi = 500000
soslow = sig.butter(10, 1000, btype='lowpass', fs = sr4, analog=False,output='sos')
soshi = sig.butter(10, 9000, btype='highpass', fs = sr4, analog=False,output='sos')

low_filtered_1 = sig.sosfilt(soslow, received_1)
low_filtered_2 = sig.sosfilt(soslow, received_2)

hi_filtered_1 = sig.sosfilt(soshi, received_1)
hi_filtered_2 = sig.sosfilt(soshi, received_2)


In [ ]:
N = np.ones(4400)
lowfilter = np.abs(ifft(sig.sosfilt(soslow,ifft(N))))
highfilter = np.abs(ifft(sig.sosfilt(soshi,ifft(N))))
lowfiltered = np.abs(ifft(low_filtered_1))
highfiltered = np.abs(ifft(hi_filtered_1))
spectrum = np.abs(ifft(received_1))
equalized_spectrum = np.abs(ifft(equalize(received_1)))

data = {
    'frequency (kHz)': np.arange(2200)/2.2e6*sr4/2,
    'lowpass': lowfilter[:2200]/max(lowfilter),
    'highpass': highfilter[:2200]/max(lowfilter),
    'lowfiltered': lowfiltered[:2200]/max(lowfiltered),
    'highfiltered': highfiltered[:2200]/max(highfiltered),
    'signal': spectrum[:2200]/max(spectrum),
    "eq": equalized_spectrum[:2200]/max(equalized_spectrum)
}
df = pd.DataFrame(data)
fig = px.line(df,'frequency (kHz)',["highfiltered","lowfiltered"],labels={"value":"intensity"})
fig.update_layout(
    font_family="Times New Roman",
    font_color="black",
    font_size=25,
    autosize=False,
    width=600,
    height=600,
    showlegend=False)

fig.show()

In [ ]:
data = {
   'time':np.arange(time_duration)/sr4*1000,
   #'amplitude0':received_0,
   'amplitude1':received_1,
   'amplitude2':received_2,
   #'amplitude3':received_3,
   #'lo_1': low_filtered_1,
   #'hi_1': hi_filtered_1,
}
df = pd.DataFrame(data)

In [ ]:
fig = px.line(df,x='time',y=['amplitude1','amplitude2'],labels={"time": "time (ms)","value":"signal amplitude"})
fig.update_layout(
    font_family="Times New Roman",
    font_color="black",
    font_size=35,
    showlegend=False)
fig.show()

In [ ]:
data = {
   'time':(np.arange(time_duration)-time_duration/2)/sr4*1000,
   'amplitude0':received_0,
   'amplitude1':received_1,
   'amplitude2':received_2,
   'amplitude3':received_3,
    'corr12': sig.correlate(received_1,received_2,mode="same"),
   'corr01': sig.correlate(received_0,received_1,mode="same"),
   'corr02': sig.correlate(received_0,received_2,mode="same"),
   'cross-correlation (unfiltered)': sig.correlate(received_1,received_2,mode="same")/max(sig.correlate(received_1,received_2,mode="same")),
   'cross-correlation (equalized)': sig.correlate(equalize(received_1),equalize(received_2),mode="same")/max(sig.correlate(equalize(received_1),equalize(received_2),mode="same")),
   'cross-correlation (lowpass)': sig.correlate(low_filtered_1,low_filtered_2,mode="same")/max(sig.correlate(low_filtered_1,low_filtered_2,mode="same")),
    'cross-correlation (highpass)': sig.correlate(hi_filtered_1,hi_filtered_2,mode="same")/max(sig.correlate(hi_filtered_1,hi_filtered_2,mode="same")),
    'corr03': sig.correlate(received_0,received_3,mode="same") 
}
df = pd.DataFrame(data)

In [ ]:
fig = px.line(df,x='time',y=['corr12'],labels={"time": "timeshift (ms)","value": "cross-correlation"})
#fig.update_layout(xaxis = dict(tickmode = 'linear',tick0 = 0,dtick = 100))
fig.update_layout(
    font_family="Times New Roman",
    font_color="black",
    font_size=35,
    showlegend=False)
fig.show()


In [ ]:
fig = px.line(df,x='time',y=['cross-correlation (highpass)','cross-correlation (lowpass)','cross-correlation (unfiltered)'],labels={"time": "timeshift (ms)","value": "cross-correlation"})
#fig.update_layout(xaxis = dict(tickmode = 'linear',tick0 = 0,dtick = 100))
fig.update_layout(
    font_family="Times New Roman",
    font_color="black",
    font_size=25,
    showlegend=True)
fig.show()


In [ ]:
50/22.05

### np.argmax(sig.correlate(received_1,received_2,mode="same"))

In [ ]:
ipd.Audio(received_1,rate=sr1)

In [ ]:
ipd.Audio(received_2,rate=sr)

In [ ]:
transform = fft(signal8[:20000])
fig = px.line(np.abs(transform)[:len(transform)//2])
fig.show()

In [ ]:
1/48e3*343

# Robustness Tests/Parameter Study

## Doppler shift

In [ ]:
def sample_doppler(factors,Nsim,resample=1):
    lo = 20
    hi = 2000
    diff = []
    difff = []
    rel_height = []
    rel_heightf = []
    for f in factors:
        d = []
        df = []
        rh = []
        rhf = []
        
        for i in range(int(Nsim)):
            
            N = 5000
            t = np.arange(0,N,1)
            T = 100
            signal8 = y4
            A = .01
            noise1 = np.random.normal(0,1,N)
            noise1 = A*noise1/max(noise1)
            noise2 = np.random.normal(0,1,N)
            noise2 = A*noise2/max(noise2)

            source = signal8/max(signal8)
            source_shifted = sig.resample(source,int(len(source)/f))
            t0 = int(sr4*5)
            dt = int(sr4*0.2)
            td = sr4//50
            received_1 = source[t0:t0+dt]+noise1[:dt]
            received_2 = source_shifted[t0+td:t0+td+dt]+noise2[:dt]
            #filtered_1 = butter_filter(received_1,[lo,hi],sr1,10)
            #filtered_2 = butter_filter(received_2,[lo,hi],sr1,10)
            corr = sig.correlate(received_1,received_2,mode="same")
            maxima = corr[sig.argrelmax(corr)[0]]
            maxima_sorted = np.sort(maxima)
            max_cross = np.argmax(corr)
            relative_height = maxima_sorted[-1]/maxima_sorted[-2]
            d.append(max_cross-dt//2)
            rh.append(relative_height)
            
            corr = sig.correlate(equalize(received_1),equalize(received_2),mode="same")
            maxima = corr[sig.argrelmax(corr)[0]]
            maxima_sorted = np.sort(maxima)
            max_cross = np.argmax(corr)
            relative_height = maxima_sorted[-1]/maxima_sorted[-2]
            df.append(max_cross-dt//2)
            rhf.append(relative_height)
        rel_height.append(np.mean(rh))
        rel_heightf.append(np.mean(rhf))
        diff.append(np.mean(d))
        difff.append(np.mean(df))
    data = {
   'f':factors,
   'unfiltered':np.array(diff)/sr4*1000,
   'equalized':np.array(difff)/sr4*1000,
   'relative strength (unfiltered)':rel_height,
    'relative strength (equalized)': rel_heightf
    }
    return pd.DataFrame(data)
    
    return data

In [ ]:
out = sample_doppler(np.linspace(0.99,1.01,50),5)

In [ ]:
fig = px.line(out,x='f',y=['relative strength (unfiltered)','relative strength (equalized)'],log_y=False,labels={'f': '$\\alpha$','value':'relative strength'})

fig.update_layout(
    font_family="Times New Roman",
    font_color="black",
    font_size=25,
    #autosize=False,
    #width=800,
    #height=500,
    showlegend=False)

fig.show()

In [ ]:
N = 500000
t = np.arange(0,N,1)
T = 10
A = 0.01
signal0 = np.zeros(N)
signal1 = A*(np.sin(t/T))
signal5 = y1
signal6 = y2
signal7 = y3
signal8 = y4
noise = np.random.normal(0,1,N)
noise1 = np.random.normal(0,1,N)
noise2 = np.random.normal(0,1,N)

source = signal8/max(signal8)
source_shifted = sig.resample(source,int(len(source)/1.01))

t0 = 15000
dt = 10000
td = 100
received_1 = source[t0:t0+dt]+A*noise1[t0:t0+dt]
received_2 = source_shifted[t0+td:t0+td+dt]+A*noise2[t0:t0+dt]

In [ ]:
fig = px.line(np.abs(fft(received_1)),log_x=True)
fig.add_scatter(y=np.abs(fft(received_2)))
fig.show()

In [ ]:
px.line(sig.correlate(np.abs(fft(received_1)),np.abs(fft(received_2)),mode="same"))

In [ ]:
data = {
   'time':np.arange(dt),
   'amplitude1':received_1,
   'amplitude2':received_2
}
df = pd.DataFrame(data)

In [ ]:
fig = px.line(df,x='time',y='amplitude1')
fig.add_scatter(x=df['time'],y=df['amplitude2'])
fig.show()

In [ ]:
fig = px.line(sig.correlate(received_1,received_2,mode="same"))
fig.show()

In [ ]:
ipd.Audio(received_1,rate=sr1)

In [ ]:
data = sample_doppler(np.logspace(-6,-1,20),5)
data_res = sample_doppler(np.logspace(-6,-1,20),5,.5)
data_res2 = sample_doppler(np.logspace(-6,-1,20),5,2)

In [ ]:
fig = px.line(data,x='factors',y='diff',log_x=True)
fig.add_scatter(x=data_res['factors'],y=data_res['diff'])
fig.add_scatter(x=data_res2['factors'],y=data_res2['diff'])

In [ ]:
fig = px.line(data,x='factors',y='rel_height',log_x=True)
fig.add_scatter(x=data_res['factors'],y=data_res['rel_height'])
fig.add_scatter(x=data_res2['factors'],y=data_res2['rel_height'])

In [ ]:
def equalize(signal):
    ft = fft(signal)
    eq_ft = []
    for i in ft:
        if abs(i) > 1e-4:
            eq_ft.append(i/abs(i))
        else:
            eq_ft.append(i)
    return(np.real(ifft(eq_ft)))

## Amplitude of noise

In [ ]:
sr4/50

In [ ]:
def sample_A(As,Nsim):
    lo = 20
    hi = 2000
    diff = []
    difff = []
    rel_height = []
    rel_heightf = []
    for A in As:
        d = []
        df = []
        rh = []
        rhf = []
        
        for i in range(int(Nsim)):
            
            N = 5000
            t = np.arange(0,N,1)
            T = 100
            signal8 = y4
            noise1 = np.random.normal(0,1,N)
            noise1 = A*noise1/max(noise1)
            noise2 = np.random.normal(0,1,N)
            noise2 = A*noise2/max(noise2)

            source = signal8/max(signal8)
            t0 = int(sr4*5)
            dt = int(sr4*0.2)
            td = sr4//50
            received_1 = source[t0:t0+dt]+noise1[:dt]
            received_2 = source[t0+td:t0+td+dt]+noise2[:dt]
            #filtered_1 = butter_filter(received_1,[lo,hi],sr1,10)
            #filtered_2 = butter_filter(received_2,[lo,hi],sr1,10)
            corr = sig.correlate(received_1,received_2,mode="same")
            maxima = corr[sig.argrelmax(corr)[0]]
            maxima_sorted = np.sort(maxima)
            max_cross = np.argmax(corr)
            relative_height = maxima_sorted[-1]/maxima_sorted[-2]
            d.append(max_cross-dt//2)
            rh.append(relative_height)
            
            corr = sig.correlate(equalize(received_1),equalize(received_2),mode="same")
            maxima = corr[sig.argrelmax(corr)[0]]
            maxima_sorted = np.sort(maxima)
            max_cross = np.argmax(corr)
            relative_height = maxima_sorted[-1]/maxima_sorted[-2]
            df.append(max_cross-dt//2)
            rhf.append(relative_height)
        rel_height.append(np.mean(rh))
        rel_heightf.append(np.mean(rhf))
        diff.append(np.mean(d))
        difff.append(np.mean(df))
    data = {
   'A':As,
    'SNR (dB)': np.log(1/As)*10,
   'unfiltered':np.array(diff)/sr4*1000,
   'equalized':np.array(difff)/sr4*1000,
   'relative strength (unfiltered)':rel_height,
    'relative strength (equalized)': rel_heightf
    }
    return pd.DataFrame(data)

In [ ]:
out = sample_A(np.logspace(-1,1,50),5)

In [ ]:
fig = px.line(out,x='SNR (dB)',y=['relative strength (unfiltered)','relative strength (equalized)'],log_y=False,labels={'value':'relative strength'})

fig.update_layout(
    font_family="Times New Roman",
    font_color="black",
    font_size=25,
    autosize=False,
    width=800,
    height=500,
    showlegend=False)

fig.show()

In [ ]:
fig = px.line(out,x='SNR (dB)',y=['unfiltered','equalized'],log_y=False,labels={'value':'predicted shift (ms)'})

fig.update_layout(
    font_family="Times New Roman",
    font_color="black",
    font_size=25,
    autosize=False,
    width=800,
    height=500,
    showlegend=False)

fig.show()

In [ ]:
px.line(out,x='A',y="rel_height",log_x=True)

In [ ]:
def plotly_dual_axis(data1,data2, title="", y1="", y2=""):
    # Create subplot with secondary axis
    subplot_fig = make_subplots(specs=[[{"secondary_y": True}]])

    #Put Dataframe in fig1 and fig2
    fig1 = px.line(data1)
    fig2 = px.line(data2)
    #Change the axis for fig2
    fig2.update_traces(yaxis="y2")

    #Add the figs to the subplot figure
    subplot_fig.add_traces(fig1.data + fig2.data)

    #FORMAT subplot figure
    subplot_fig.update_layout(title=title, yaxis=dict(title=y1), yaxis2=dict(title=y2))

    #RECOLOR so as not to have overlapping colors
    subplot_fig.for_each_trace(lambda t: t.update(line=dict(color=t.marker.color)))


    return subplot_fig

## Presence of unwanted noise signal

In [ ]:
def sample_unwanted(As,Nsim,filt=False):
    diff2 = []
    rel_height2 = []
    rel_height1 = []
    diff1 = []
    lo = 20
    hi = 2000
    for A in As:

        for i in range(int(Nsim)):
            rh1 = []
            rh2 = []
            d1 = []
            d2 = []
            N = 50000
            t = np.arange(0,N,1)
            signal8 = y4
            noise1 = .1*np.random.normal(0,1,N)
            noise2 = .1*np.random.normal(0,1,N)
            noise = A*np.random.normal(0,1,N)
            source = signal8/max(signal8)

            t0 = 15000
            td = 100
            td_noise=200
            dt = 2000
            received_1 = source[t0:t0+dt]+noise1[t0:t0+dt]+noise[t0:t0+dt]
            received_2 = source[t0+td:t0+td+dt]+noise2[t0:t0+dt]+noise[t0+td_noise:t0+td_noise+dt]
            if filt:
                received_1 = equalize(received_1)
                received_2 = equalize(received_2)
            corr = sig.correlate(received_1,received_2,mode="same")
            arm = sig.argrelmax(corr)[0]
            maxima = corr[arm]
            max_cross1 = arm[np.argmax(maxima)]
            max_cross2 = arm[np.argsort(maxima)[-2]]
            relative_height1 = maxima[np.argmax(maxima)]/maxima[np.argsort(maxima)[-2]]
            relative_height2 = maxima[np.argsort(maxima)[-2]]/maxima[np.argsort(maxima)[-3]]
            d1.append(max_cross1-dt//2)
            d2.append(max_cross2-dt//2)
            rh1.append(relative_height1)
            rh2.append(relative_height2)
        rel_height1.append(np.mean(rh1))
        rel_height2.append(np.mean(rh2))
        diff1.append(np.mean(d1))
        diff2.append(np.mean(d2))
        
    display(len(rel_height1))
    data = {
   'A':As,
   'first peak':diff1,
   'second peak':diff2,
   'rel_height1':rel_height1,
    'rel_height2': rel_height2
    }
    
    out = pd.DataFrame(data)
    
    return out

In [ ]:
out = sample_unwanted(np.logspace(-2,1,20),5,True)

In [ ]:
fig = px.line(out,x='A',y=['first peak','second peak'],log_x=True,labels={'value':'cross correlation index'})
fig.update_layout(yaxis_range=[0,300])
fig.show()

In [ ]:
fig = px.line(out,x='A',y=['rel_height1','rel_height2'],log_x=True)
#fig.add_scatter(x=out_filt['A'],y=out_filt['rel_height1'],name='First peak - filtered')
#fig.add_scatter(x=out_filt['A'],y=out_filt['rel_height2'],name='Second peak - filtered')
fig.show()

In [ ]:
N = 50000
t = np.arange(0,N,1)
T = 10
A = 1

filt = True
lo, hi = 50,1000

signal0 = np.zeros(N)
signal1 = A*(np.sin(t/T))
signal5 = y1
signal6 = y2
signal7 = y3
signal8 = y4
noise = A*np.random.normal(0,1,N)
noise1 = .5*np.random.normal(0,1,N)
noise2 = .5*np.random.normal(0,1,N)

source = signal8/max(signal8)

t0 = 15000
dt = 2000
td = 100
td_noise = 200
received_1 = source[t0:t0+dt]+noise1[t0:t0+dt]+noise[t0:t0+dt]
received_2 = source[t0+td:t0+td+dt]+noise2[t0:t0+dt]+noise[t0+td_noise:t0+td_noise+dt]
if filt:
    received_1 = butter_filter(received_1,lo,hi,sr1,10)
    received_2 = butter_filter(received_2,lo,hi,sr1,10)

In [ ]:
fig = px.line(sig.correlate(received_1,received_2,mode="same"))
fig.show()

In [ ]:
maxima = sig.correlate(received_1,received_2,mode="same")
maxima[np.argmax(maxima)]

## Sample Length (uncorrelated noise)

In [ ]:
def sample_dt(dts,Nsim):
    lo = 20
    hi = 2000
    diff = []
    difff = []
    rel_height = []
    rel_heightf = []
    for dt in dts:
        dt = int(dt)
        d = []
        df = []
        rh = []
        rhf = []
        
        for i in range(int(Nsim)):
            N = 50000
            T = 100
            A = 0.1
            signal8 = y4
            noise1 = A*np.random.normal(0,1,N)
            noise2 = A*np.random.normal(0,1,N)

            source = signal8/max(signal8)
            t0 = int(sr4*5)
            td = sr4//50
            received_1 = source[t0:t0+dt]+noise1[:dt]
            received_2 = source[t0+td:t0+td+dt]+noise2[:dt]
            #filtered_1 = butter_filter(received_1,[lo,hi],sr1,10)
            #filtered_2 = butter_filter(received_2,[lo,hi],sr1,10)
            corr = sig.correlate(received_1,received_2,mode="same")
            maxima = corr[sig.argrelmax(corr)[0]]
            maxima_sorted = np.sort(maxima)
            max_cross = np.argmax(corr)
            relative_height = maxima_sorted[-1]/maxima_sorted[-2]
            d.append(max_cross-dt//2)
            rh.append(relative_height)
            
            corr = sig.correlate(equalize(received_1),equalize(received_2),mode="same")
            maxima = corr[sig.argrelmax(corr)[0]]
            maxima_sorted = np.sort(maxima)
            max_cross = np.argmax(corr)
            relative_height = maxima_sorted[-1]/maxima_sorted[-2]
            df.append(max_cross-dt//2)
            rhf.append(relative_height)
        rel_height.append(np.mean(rh))
        rel_heightf.append(np.mean(rhf))
        diff.append(np.mean(d))
        difff.append(np.mean(df))
    data = {
   'dt':dts/sr4*1000,
   'diff':np.array(diff)/sr4*1000,
   'difff':np.array(difff)/sr4*1000,
   'rel_height':rel_height,
    'rel_heightf': rel_heightf
    }
    return pd.DataFrame(data)

In [ ]:
out = sample_dt(np.logspace(2,3.5,200),5)

In [ ]:
fig = px.line(out,x='dt',y=['diff','difff'],log_x=False,labels={'dt':'sampling time (ms)','value': 'timeshift (ms)'})
fig.update_layout(
    font_family="Times New Roman",
    font_color="black",
    font_size=25,
    autosize=False,
    width=800,
    height=500,
    showlegend=False)
fig.show()

In [ ]:
fig = px.line(out,x='dt',y=['rel_height','rel_heightf'],log_x=False,labels={'dt':'sampling time (ms)','value': 'relative strength'})
fig.update_layout(
    font_family="Times New Roman",
    font_color="black",
    font_size=25,
    autosize=False,
    width=800,
    height=500,
    showlegend=False)
fig.show()

# Combining length and noise

In [ ]:
def sample(As,dts,Nsim):
    out1 = np.zeros([len(dts),len(As)])
    out2 = np.zeros([len(dts),len(As)])

    for i in range(len(As)):
        A = As[i]
        for j in range(len(dts)):
            dt = int(dts[j])
            diff = []
            rel_height = []
            for k in range(int(Nsim)):
                N = 100000
                t = np.arange(0,N,1)
                T = 1000
                signal8 = y4
                noise1 = np.random.normal(0,1,N)
                noise1 = A*noise1/max(noise1)
                noise2 = np.random.normal(0,1,N)
                noise2 = A*noise2/max(noise2)

                source = signal8/max(signal8)

                t0 = int(sr4*5)
                td = sr4//50
                received_1 = source[t0:t0+dt]+noise1[:dt]
                received_2 = source[t0+td:t0+td+dt]+noise2[:dt]
                corr = sig.correlate(received_1,received_2,mode="same")
                maxima = corr[sig.argrelmax(corr)[0]]
                maxima_sorted = np.sort(maxima)
                max_cross = np.argmax(corr)
                relative_height = maxima_sorted[-1]/maxima_sorted[-2]
                rel_height.append(relative_height)
            out1[i,j] = np.mean(rel_height)
            if int(np.mean(max_cross-dt//2))==td:
                out2[i,j] = 1
            else:
                out2[i,j] = 0
    return [out1,out2]

In [ ]:
sampling_times = np.linspace(200,20000,40)
amplitudes = np.logspace(-3,2,40)
data = sample(amplitudes,sampling_times,1)

In [ ]:
strong_signal = np.zeros([len(data[0]),len(data[0][0])])
for i in range(len(data[0])):
    for j in range(len(data[0][0])):
        if data[0][i][j] > 1.5:
            strong_signal[i][j] = 2+np.log(data[0][i][j]-.5)
        else:
            strong_signal[i][j] = data[0][i][j]

In [ ]:
fig = px.imshow(strong_signal,labels=dict(x="sampling time (10ms)", y="SNR (dB)", color="signal strength"),
                x=sampling_times/sr4*100,
                y=np.log10(1/amplitudes)*10,aspect='auto')
fig.update_layout(font_family="Times New Roman",
    font_color="black",
    font_size=25,
    autosize=True,
    width=1000,
    height=500,
    showlegend=False,
    yaxis = dict(scaleanchor = 'x'),
                 plot_bgcolor='rgba(0,0,0,0)',coloraxis_showscale=True)
fig.show()

In [ ]:
fig = px.imshow(data[1],labels=dict(x="sampling time (10ms)", y="SNR (dB)", color="correct shift"),
                x=sampling_times/sr4*100,
                y=np.log10(1/amplitudes)*10,aspect='auto')
fig.update_layout(font_family="Times New Roman",
    font_color="black",
    font_size=25,
    autosize=True,
    width=1000,
    height=500,
    showlegend=False,
    yaxis = dict(scaleanchor = 'x'),
                 plot_bgcolor='rgba(0,0,0,0)',coloraxis_showscale=True)
fig.show()

## Doppler and Sampling

In [ ]:
def sample(factors,Nsim):
    out1 = np.zeros([len(factors)])
    out2 = np.zeros([len(factors)])

    for i in range(len(factors)):
        f = factors[i]
        dt = sr4//5
        diff = []
        rel_height = []
        for k in range(int(Nsim)):
            N = 40000
            t = np.arange(0,N,1)
            T = 1000
            signal8 = y4
            A = .01
            noise1 = A*np.random.normal(0,1,N)
            noise2 = A*np.random.normal(0,1,N)

            source = signal8/max(signal8)
            source_shifted = sig.resample(source,int(len(source)*f))

            t0 = int(sr4*5)
            td = sr4//50
            received_1 = source[t0:t0+dt]+noise1[:dt]
            received_2 = source_shifted[t0+td:t0+td+dt]+noise2[:dt]
            corr = sig.correlate(equalize(received_1),equalize(received_2),mode="same")
            maxima = corr[sig.argrelmax(corr)[0]]
            maxima_sorted = np.sort(maxima)
            max_cross = np.argmax(corr)
            relative_height = maxima_sorted[-1]/maxima_sorted[-2]
            rel_height.append(relative_height)
        out1[i] = np.mean(rel_height)
        out2[i] = int(np.mean(max_cross-dt//2))-td
    return [out1,out2] 

In [ ]:
shifts = np.linspace(0.99,1.01,20)
data = sample(shifts,10)

In [ ]:
px.line(x=shifts,y=data[1])

In [ ]:
def sample(factors,dts,Nsim):
    out1 = np.zeros([len(dts),len(factors)])
    out2 = np.zeros([len(dts),len(factors)])

    for i in range(len(factors)):
        f = factors[i]
        for j in range(len(dts)):
            dt = int(dts[j])
            diff = []
            rel_height = []
            for k in range(int(Nsim)):
                N = 200000
                t = np.arange(0,N,1)
                T = 1000
                signal8 = y4
                A = .01
                noise1 = A*np.random.normal(0,1,N)
                noise2 = A*np.random.normal(0,1,N)

                source = signal8/max(signal8)
                source_shifted = sig.resample(source,int(len(source)/(1+f)))
                
                t0 = 15000
                td = 100
                received_1 = source[t0:t0+dt]+noise1[t0:t0+dt]
                received_2 = source_shifted[t0+td:t0+td+dt]+noise2[t0:t0+dt]
                corr = sig.correlate(received_1,received_2,mode="same")
                maxima = corr[sig.argrelmax(corr)[0]]
                maxima_sorted = np.sort(maxima)
                max_cross = np.argmax(corr)
                relative_height = maxima_sorted[-1]/maxima_sorted[-2]
                rel_height.append(relative_height)
            out1[i,j] = np.mean(rel_height)
            out2[i,j] = abs(int(np.mean(max_cross-dt//2))-td)
    return [out1,out2]

In [ ]:
shifts = np.logspace(-6,0,30)
sampling_times = np.logspace(2,5,30)
data = sample(shifts,sampling_times,10)

In [ ]:
fig = px.imshow(data[0],labels=dict(x="doppler shift", y="sampling duration", color="signal strength"),
                x=np.log10(shifts),
                y=np.log10(sampling_times))
fig.update_xaxes(side="top")
fig.show()

In [ ]:
fig = px.imshow(np.log(1+data[1]),labels=dict(x="doppler shift", y="sampling time", color="absolute error"),
                x=np.log10(shifts),
                y=np.log10(sampling_times))
fig.update_xaxes(side="top")
fig.show()

# FILTERING

In [ ]:
def butter_filter(data, freqs, sampling_frequency, order):
    sos = sig.butter(order, freqs, btype='band', fs = sampling_frequency, analog=False,output='sos')
    y = sig.sosfilt(sos, data)
    return y

In [ ]:
x = np.ones(10000)
px.line(np.abs(fft(butter_filter(ifft(x),[100,1000],5000,10))))

In [ ]:
px.line(sos[0])

In [ ]:
N = 500000
t = np.arange(0,N,1)
f = 5000
A = .2
lo = 100
hi = 2000

signal0 = np.zeros(N)
signal1 = (np.sin(t*2*np.pi*f/sr1))
signal2 = A*(1/np.sqrt(5)*np.sin(t/T)+2/np.sqrt(5)*np.cos(3*t/T))
signal3 = A*(np.sin(np.sqrt(t)/T))
signal4 = A*(np.sin(t**2/T))
signal5 = y1
signal6 = y2
signal7 = y3
signal8 = y4
noise = np.random.normal(0,1,N)
noise1 = A*np.random.normal(0,1,N)
noise2 = A*np.random.normal(0,1,N)
noise_cauchy = np.random.standard_cauchy(N)


source = signal8/max(signal8)

time_start = 15000
time_duration = 10000
time_diff = 100
received_1 = source[time_start:time_start+time_duration]+noise1[time_start+time_diff:time_start+time_duration+time_diff]+signal1[time_start:time_start+time_duration]
received_2 = source[time_start+time_diff:time_start+time_duration+time_diff]+noise2[time_start+time_diff:time_start+time_duration+time_diff]+signal1[time_start+time_diff*2:time_start+time_duration+time_diff*2]

filtered_1 = butter_filter(received_1,[lo,hi],sr4,10)
filtered_2 = butter_filter(received_2,[lo,hi],sr4,10)

In [ ]:
data = {
   'time':np.arange(time_duration),
   'received_1':received_1,
   'received_2':received_2,
   'filtered_1': filtered_1,
   'filtered_2':filtered_2,
   'corr_1': sig.correlate(received_1,received_2,mode="same"),
   'corr_2': sig.correlate(filtered_1,filtered_2,mode="same")
}
df = pd.DataFrame(data)

In [ ]:
fig = px.line(df,x='time',y=['received_1','filtered_1'])
fig.show()

In [ ]:
fig = px.line(data,x='time',y=['corr_1','corr_2'])
fig.show()

In [ ]:
ipd.Audio(filtered_1,rate=sr1)

In [ ]:
a = np.zeros([2,5])
a[1,0]

# CONDITIE

In [ ]:
mics = [(0,0,0),(1,0,0),(0,1,0),(0,0,1)]
point = (5,5,5)
def rel_distances(points,mic_pos = [(0,0,0),(1,0,0),(0,1,0),(0,0,1)]):
    res = np.zeros([len(points),len(mic_pos)-1])
    for i in range(len(points)):
        distances = []
        for j in range(len(mic_pos)):
            distances.append(np.linalg.norm(np.array(points[i])-np.array(mic_pos[j])))
        for j in range(len(distances)-1):
            res[i,j] = distances[0]-distances[j+1]
    return res

In [ ]:
point = np.array([10,10,10])
eps = 0.1
np.mean(condition_test([point],eps,10000),axis=0)/eps

In [ ]:
cond = []
eps = .01
t = np.linspace(2,20,25)
for i in t:
    cond.append(np.mean(condition_test([np.array([i,i,i])/np.sqrt(3)],eps,1000))/eps)

In [ ]:
px.line(x=t,y=1/np.array(cond))

In [ ]:
def random_three_vector():
    phi = np.random.uniform(0,np.pi*2)
    costheta = np.random.uniform(-1,1)

    theta = np.arccos( costheta )
    x = np.sin( theta) * np.cos( phi )
    y = np.sin( theta) * np.sin( phi )
    z = np.cos( theta )
    return np.array([x,y,z])

def condition_test(point,epsilon,N=1000):
    diffs = []
    for i in range(N):
        diffs.append(abs(rel_distances(point)-rel_distances(point+epsilon*random_three_vector())))
    return np.array(diffs)

In [ ]:
rel_distances(point)

In [ ]:
N = 50000
t = np.arange(0,N,1)
signal8 = y4
A = 0
f = .1
noise1 = A*np.random.normal(0,1,N)
noise2 = A*np.random.normal(0,1,N)

source = signal8/max(signal8)
source_shifted = sig.resample(source,int(len(source)*(1+f)))
                
t0 = int(sr4*1)
dt = int(sr4)
td = sr4//50

received_1 = source[t0:t0+dt]
received_2 = source_shifted[t0:t0+dt]

spectrum_1 = np.abs(fft(received_1))[:dt//2]
spectrum_2 = np.abs(fft(received_2))[:dt//2]

freqs = np.array(range(dt//2))
logfreqs = np.logspace(0,np.log10(dt//2),10000)

logspectrum_1 = np.interp(logfreqs,freqs,spectrum_1)
logspectrum_2 = np.interp(logfreqs,freqs,spectrum_2)

In [ ]:
dt//2

In [ ]:
fig = px.line(x=np.linspace(0,1,10000)*np.log10(dt//2),y=logspectrum_1/max(logspectrum_1),log_x=False,labels={'y':'intensity','x':"log(f)"})
fig.add_scatter(x=np.linspace(0,1,10000)*np.log10(dt//2),y=logspectrum_2/max(logspectrum_2))
fig.update_layout(
    font_family="Times New Roman",
    font_color="black",
    font_size=25,
    autosize=False,
    width=1000,
    height=600,
    showlegend=False)

In [ ]:
fig = px.line(x=np.linspace(-.5,.5,10000)*np.log10(dt//2),y=sig.correlate(logspectrum_1,logspectrum_2,mode="same")/max(sig.correlate(logspectrum_1,logspectrum_2,mode="same")),labels={'y':'cross-correlation','x':"(log) frequency shift"})
fig.update_layout(
    font_family="Times New Roman",
    font_color="black",
    font_size=25,
    autosize=False,
    width=1000,
    height=600,
    showlegend=False)
fig.show()

In [ ]:
logfreqs[5094]/logfreqs[5000]

In [ ]:
10**(.041428)